## Storm Mode/Precipitation Type Classification Output for 3D Reflectivity from derived dBZ of WRF Simulations. 

**Based on the Convective/Stratiform separation on the 12 $\sigma$ level of reflectivity.**

**Calculate the max. composite reflectivity instead of REFLC_10CM from CONUS1 runs.**

**Storm Mode Classification starts from the Composite dBZ (Rain Area) identification, and add another mode: Shallow (Non-Deep) Convective Cores (SCC) to represent shallow conveciton.**

**Output the Storm Mode Classification information (1:DCC; 2:SCC; 3:WCC; 4:DWCC; 5:BSR) to CONUS dBZ data.**

**For [High Resolution WRF Simulations of the Current and Future Climate of North America](https://rda.ucar.edu/datasets/ds612.0/).**

**Hungjui Yu 20211029**

In [1]:
# import sys
# from shutil import copyfile
import time
import datetime as dt
# import pytz
from netCDF4 import Dataset # MFDataset
import numpy as np
from scipy.ndimage import label, generate_binary_structure
import xarray as xr
import wrf
from wrf import (getvar, interplevel, destagger)

import cartopy.crs as ccrs
import cartopy.feature as cfeat
import matplotlib as mpl
import matplotlib.pyplot as plt


**Set input files paths and names:**

In [2]:
def set_input_names(file_date):

    file_path_1_conus = '/gpfs/fs1/collections/rda/data/ds612.0'
    file_path_1_dbz = '/glade/scratch/hungjui/DATA_WRF_CONUS_1_dBZ_v1.0'
    file_path_2 = '/' + wrf_sim_type # '/CTRL3D'
    file_path_3 = '/{}'.format(file_date.strftime('%Y'))

    file_names = dict( dbz = file_path_1_dbz
                           + file_path_2 
                           + '/20130913'# file_path_3 
                           + '/wrf3d_d01_' + wrf_sim_type[0:-2] + '_dbz_{}.nc'.format(file_date.strftime('%Y%m%d'))
                       , Z = file_path_1_conus
                           + file_path_2 
                           + file_path_3 
                           + '/wrf3d_d01_' + wrf_sim_type[0:-2] + '_Z_{}.nc'.format(file_date.strftime('%Y%m%d'))
                     )
    
    return file_names

**Function: Reflectivity Geo-Height Interpolation (linearly):**

In [1]:
def dbz_geoh_interp(refl, geoH, interp_lev_km):
    
    # # Use linear Z for interpolation:
    refl_linear = 10**(refl/10.)
    
    ## Interpolation:
    ## !!! convert interpolation level to the same as geo-H (meter) !!!
    refl_linear_lev = interplevel(refl_linear, geoH, interp_lev_km*1000)
    
    ## Convert back to dBz after interpolation:
    refl_lev = 10.0 * np.log10(refl_linear_lev)
    
    return refl_lev

**Function: Set Storm Mode Classification Thresholds:**

In [3]:
def set_classification_thresholds(threshold_type):
    
    ## Make sure the thresholds are either Mocderate or Strong:
    assert threshold_type in ['moderate', 'strong']
    
    if ( threshold_type == 'moderate' ):
        dbz_threshold = 30 # dBZ
        height_threshold = 8 # km
        WCC_threshold = 800 # km^2
        BSR_threshold = 40000 # km^2
    else:
        dbz_threshold = 40 # dBZ
        height_threshold = 10 # km
        WCC_threshold = 1000 # km^2
        BSR_threshold = 50000 # km^2
        
    return dbz_threshold, height_threshold, WCC_threshold, BSR_threshold


**Function: Storm Mode Classification:**

In [2]:
def storm_mode_class( refl, reflc, CS_mask
                    , geoH
                    , resolution_WRF = 4 # km grid spacing 
                    , threshold_type
                    # , dbz_threshold # dBZ
                    # , height_threshold # km
                    # , WCC_threshold # km^2
                    # , BSR_threshold # km^2
                    ):
    
    ## ======================================================================
    ## 
    ## Run:     storm_mode_class(da_wrf_dbz, da_wrf_reflc, da_wrf_CSmask, data_wrf_z_unstag)
    ## 
    ## Input:
    ## refl:    Derived 3D reflectivity
    ## reflc:   Derived 2D max. reflectivity composite
    ## CS_mask: 2D Convective/Stratifom mask at 12th sigma level (~1.5–2km agl.)
    ## geoH:    3D Geopotential-height (msl. in meter)
    ##
    ## ======================================================================
    
    ## Set thresholds:
    dbz_threshold, height_threshold, WCC_threshold, BSR_threshold = set_classification_thresholds(threshold_type)
    
    ## Set pixel numbers required for WCC & BSR:
    WCC_pixels_required = WCC_threshold/(resolution_WRF**2)
    BSR_pixels_required = BSR_threshold/(resolution_WRF**2)
    
    ## Interpolate reflectivity to height threshold:
    refl_lev = dbz_geoh_interp(refl, geoH, height_threshold)
    
    ## Generate a structuring element that will consider features connected even if they touch diagonally:
    se = generate_binary_structure(2, 2)
    
    
    ## ======================================================================
    ## 1:
    ## Threshold - Max. Composite dBZ & Convective Mask:
    reflc_boo_tmp = np.where( (reflc >= dbz_threshold) & (CS_mask > 0), 1, 0 )
    
    ## DCC & SCC masking:
    DCC_mask = np.where( ((reflc_boo_tmp == 1) & (refl_lev >= dbz_threshold)), 1, 0 )
    SCC_mask = np.where( ((reflc_boo_tmp == 1) & (refl_lev < dbz_threshold)), 1, 0 )
    
    
    ## ======================================================================
    ## 2:
    ## WCC & DWCC masking:
    labeled_array_reflc_boo, num_features_reflc_boo = label( reflc_boo_tmp, structure=se )
    
    WCC_mask = np.zeros_like(labeled_array_reflc_boo)
    DWCC_mask = np.zeros_like(labeled_array_reflc_boo)

    for feati in np.arange(num_features_reflc_boo):
        feat_id = feati+1
        if ( (labeled_array_reflc_boo == feat_id).sum() > WCC_pixels_required ):
            if ( (DCC_mask[np.where(labeled_array_reflc_boo == feat_id)]).sum() == 0 ):
                WCC_mask = np.where( (labeled_array_reflc_boo == feat_id), 1, WCC_mask )
            else:
                DWCC_mask = np.where( (labeled_array_reflc_boo == feat_id), 1, DWCC_mask )
            
    
    ## ======================================================================
    ## 3:
    ## DCC mask adjustment for DWCC:
    DCC_mask[np.where(DWCC_mask == 1)] = 0
    
    ## SCC mask adjustment for WCC and DWCC:
    SCC_mask[np.where( (WCC_mask) | (DWCC_mask == 1) )] = 0
        
    ## ======================================================================
    ## 4:
    ## Threshold - Stratiform Mask:
    stratiform_boo_tmp = np.where( (CS_mask == 0), 1, 0 )
    
    ## BSR masking:
    labeled_array_BSR, num_features_BSR = label( stratiform_boo_tmp, structure=se )
    
    BSR_mask = np.zeros_like(labeled_array_BSR)

    for feati in np.arange(num_features_BSR):
        feat_id = feati+1
        if ( (labeled_array_BSR==feat_id).sum() > BSR_pixels_required ):
            BSR_mask = np.where( (labeled_array_BSR==feat_id), 1, BSR_mask )
    
    
    return DCC_mask, SCC_mask, WCC_mask, DWCC_mask, BSR_mask
    

**Function: Merge masks to Storm Modes:**

In [ ]:
def merge_to_Storm_Mode( DCC_mask
                       , SCC_mask
                       , WCC_mask
                       , DWCC_mask
                       , BSR_mask
                       ):
    
    Storm_Mode = np.zeros_like(DCC_mask)
    Storm_Mode = np.where( (DCC_mask==1), 1, Storm_Mode )
    Storm_Mode = np.where( (SCC_mask==1), 2, Storm_Mode )
    Storm_Mode = np.where( (WCC_mask==1), 3, Storm_Mode )
    Storm_Mode = np.where( (DWCC_mask==1), 4, Storm_Mode )
    Storm_Mode = np.where( (BSR_mask==1), 5, Storm_Mode )

    # Storm_Mode[np.where(Storm_Mode == 0)] = np.nan
    
    return Storm_Mode

### Main Function:

In [6]:
def main_function(file_date_time):
    
    ## Set file datetime:
    # file_date_time = dt.datetime(2013, 9, 13, 0, 0, 0, tzinfo=pytz.utc)
    print('\nProcessing: {}'.format(file_date_time.strftime('%Y%m%d')), end=': ')
    
    ## Set input files paths and names:
    file_name_dict = set_input_names(file_date_time)

    ## Get the 3-hourly time list:
    nc_wrf_Z = Dataset(file_name_dict['Z'], mode='r')
    wrf_3hour_list = wrf.extract_times(nc_wrf_Z, timeidx=wrf.ALL_TIMES, meta=False, do_xtime=False)
    
    ## Open dBZ data array and append calculated data:
    ds_wrf_dbz = xr.open_dataset(file_name_dict['dbz'])
    
    for hi in range(len(wrf_3hour_list)):
        
        print(str(hi) + ' | ', end=' ')

        ## Get dBZ data:
        da_wrf_dbz = ds_wrf_dbz['dBZ'].isel(Time=hi)
        da_wrf_CSmask = ds_wrf_dbz['CS_mask'].isel(Time=hi)

        ## Calculate the max. composite dBZ:
        da_wrf_reflc = da_wrf_dbz.max(dim='bottom_top')

        ## Get geopotential height:
        data_wrf_z_unstag = wrf.destagger(getvar(nc_wrf_Z), 'Z', timeidx=hi, meta=False), 0)
        
        ## Storm Mode Classification (moderate thresholds):
        dbz_threshold, height_threshold, WCC_threshold, BSR_threshold = 
        
        DCC_mask, SCC_mask, WCC_mask, DWCC_mask, BSR_mask = storm_mode_class( da_wrf_dbz
                                                                            , da_wrf_reflc
                                                                            , da_wrf_CSmask
                                                                            , data_wrf_z_unstag
                                                                            , 'moderate'
                                                                            )
        Storm_Mode_single_m = merge_to_Storm_Mode( DCC_mask, SCC_mask, WCC_mask, DWCC_mask, BSR_mask )
        
        ## Storm Mode Classification (strong thresholds):
        dbz_threshold, height_threshold, WCC_threshold, BSR_threshold = 
        
        DCC_mask, SCC_mask, WCC_mask, DWCC_mask, BSR_mask = storm_mode_class( da_wrf_dbz
                                                                            , da_wrf_reflc
                                                                            , da_wrf_CSmask
                                                                            , data_wrf_z_unstag
                                                                            , 'strong'
                                                                            )
        Storm_Mode_single_s = merge_to_Storm_Mode( DCC_mask, SCC_mask, WCC_mask, DWCC_mask, BSR_mask )
        
        ## Stack the Storm Mode according to hours:
        if ( hi == 0 ):
            Storm_Mode_m = np.expand_dims(Storm_Mode_single_m, axis=0)
            Storm_Mode_s = np.expand_dims(Storm_Mode_single_s, axis=0)
        else:
            Storm_Mode_m = np.append(Storm_Mode_m, np.expand_dims(Storm_Mode_single_m, axis=0), axis=0)
            Storm_Mode_s = np.append(Storm_Mode_s, np.expand_dims(Storm_Mode_single_s, axis=0), axis=0)
            
    ## Add Storm Mode to dBZ dataset:
    ds_wrf_dbz['Storm_Mode_m'] = (['Time', 'south_north', 'west_east'], Storm_Mode_m)
    ds_wrf_dbz['Storm_Mode_s'] = (['Time', 'south_north', 'west_east'], Storm_Mode_s)
    
    ds_wrf_dbz.close()
    
    nc_wrf_Z.close()
    
    return ds_wrf_dbz


### Main Program:

In [7]:
start = time.time()


## WRF Model Simulation Category:
wrf_sim_type = 'CTRL3D'
# wrf_sim_type = 'PGW3D'

## Loop through a period:
target_date_range = pd.date_range(start='2013-9-13', end='2013-9-13', tz=pytz.utc)

for dayi in target_date_range:
        
    ## Derive Convective/Stratiform mask into dBZ dataset:
    ds_wrf_dbz = main_function(dayi)
    
    ## Add attributes to Storm Mode:
    ds_wrf_dbz.Storm_Mode_m.attrs['long_name'] = 'Storm Mode (moderate thresholds)'
    ds_wrf_dbz.Storm_Mode_m.attrs['description'] = 'Classified Storm Modes with moderate thresholds (1:DCC; 2:SCC; 3:WCC; 4:DWCC; 5:BSR)'
    
    ds_wrf_dbz.Storm_Mode_s.attrs['long_name'] = 'Storm Mode (strong thresholds)'
    ds_wrf_dbz.Storm_Mode_s.attrs['description'] = 'Classified Storm Modes with strong thresholds (1:DCC; 2:SCC; 3:WCC; 4:DWCC; 5:BSR)'
    
    ## Set output file path and name (to the original dBZ dataset):
    file_name_dict = set_input_names(dayi)
    file_path_name = file_name_dict['dbz']
    
    ds_wrf_dbz.to_netcdf(file_path_name, 'a')
    ds_wrf_dbz.close()

    
end = time.time()

print("RUNTIME：%f SEC" % (end - start))
print("RUNTIME：%f MIN" % ((end - start)/60))
print("RUNTIME：%f HOUR" % ((end - start)/3600))
    


Processing: 20130912: 0 |  1 |  2 |  3 |  4 |  5 |  6 |  7 |  RUNTIME：72.465595 SEC
RUNTIME：1.207760 MIN
RUNTIME：0.020129 HOUR


In [8]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


<font color='teal'>**Supplement Codes:**</font>

In [9]:
# from netCDF4 import (Dataset, MFDataset)
# nc_wrf_dbz = Dataset( '/glade/scratch/hungjui/DATA_WRF_CONUS_1_dBZ_v1.0/CTRL3D/20130913/wrf3d_d01_CTRL_dbz_20130913.nc'
#                     , mode='a')
# # nc_wrf_dbz.createDimension('test_dim', 1)
# print(nc_wrf_dbz)
# nc_wrf_dbz.close()